# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [30]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [31]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,Data Analyst,Live Nation Entertainment,"Los Angeles, CA"
1,Data Analyst,FRONT,"San Francisco, CA"
2,Data Analyst,Front,"San Francisco, CA"
3,Data Analyst,Tesla,"Fremont, CA"
4,Data Analyst,"Goldstone Partners, Inc.","Boulder, CO"
5,Data Analyst,Banyan Health Systems,Miami Metropolitan Area
6,"Data Analyst, Business Analysis and Reporting ...",Publix Super Markets,"Lakeland, FL"
7,Data Analyst,Bradford & Galt,"St Louis, MO"
8,Fraud Data Analyst,Abercrombie & Fitch Co.,"Columbus, OH"
9,Data Analytics Consultant,INNOVATO Solutions,"San Francisco, CA"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [82]:
def scrape_linkedin_job_search(keywords, num_pages):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    scrape_urls = []
    # Assemble the full url with parameters
    for x in range(num_pages):
        if x == 0:
            scrape_urls.append(''.join([BASE_URL, 'keywords=', keywords]))
        else:
            scrape_urls.append(''.join([BASE_URL, 'keywords=', keywords, '&start=', str(num_pages * 25)]))
            
    for scrape_url in scrape_urls:

    # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        if page.status_code == requests.codes.ok:
            soup = BeautifulSoup(page.text, 'html.parser')
        else:
            break

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    print(scrape_url)
    # Return dataframe
    return data
    

results = scrape_linkedin_job_search('data%20analysis', 2)
results


https://www.linkedin.com/jobs/search/?keywords=data%20analysis&start=50


,Title,Company,Location
0,Data Analyst,Live Nation Entertainment,"Los Angeles, CA"
1,Data Analyst,FRONT,"San Francisco, CA"
2,Data Analyst,Front,"San Francisco, CA"
3,Data Analyst,Tesla,"Fremont, CA"
4,Data Analyst,"Goldstone Partners, Inc.","Boulder, CO"
5,Data Analyst,Banyan Health Systems,Miami Metropolitan Area
6,"Data Analyst, Business Analysis and Reporting ...",Publix Super Markets,"Lakeland, FL"
7,Data Analyst,Bradford & Galt,"St Louis, MO"
8,Fraud Data Analyst,Abercrombie & Fitch Co.,"Columbus, OH"
9,Data Analytics Consultant,INNOVATO Solutions,"San Francisco, CA"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [85]:
def scrape_linkedin_job_search(keywords, country):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&location=', country])
    

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    print(scrape_url)
    # Return dataframe
    return data


In [89]:
results = scrape_linkedin_job_search('data%20analysis', 'München')
results

https://www.linkedin.com/jobs/search/?keywords=data%20analysis&location=München


,Title,Company,Location
0,Data Analyst,Amazon,"Munich, Bavaria, Germany"
1,(Junior) Data Scientist / Data Analyst (w/m/d),KPMG Deutschland,"Munich, Bavaria, Germany"
2,Data Analyst / Data Engineer (m/w/d) - uptodate,Bridgemaker GmbH,"Munich, Bavaria, Germany"
3,Data Analyst (m/f/x),AutoScout24,"Munich, Bavaria, Germany"
4,Data Scientist (m/f/d),Huawei,"Munich, Bavaria, Germany"
5,Intern Market Research & Data Analysis (m/f/x),Swiss Re,"Munich, Bavaria, Germany"
6,Data Analyst (m/w/d),IGEL Projekt Service GmbH,"Munich, Bavaria, Germany"
7,Data Analyst (m/f/d),Allianz Global Corporate & Specialty (AGCS),"Munich, Bavaria, Germany"
8,Data Scientist (m/f/d),FlixBus,"Munich, Bavaria, Germany"
9,Data Analyst,Allianz SE (global headquarters),"Munich, Bavaria, Germany"


## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [72]:
# your code here
def scrape_linkedin_job_search(keywords, country, num_days):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    seconds = num_days * 86400 
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&location=', country, '&f_TPR=r', str(seconds)])
    

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    print(scrape_url)
    
    # Return dataframe
    return data


In [73]:
results = scrape_linkedin_job_search('data%20analysis', 'Brazil', 3)
results

https://www.linkedin.com/jobs/search/?keywords=data%20analysis&location=Brazil&f_TPR=r259200


,Title,Company,Location
0,Analista Data Analytics,PwC Brasil,"São Paulo, São Paulo, Brazil"
1,Data Analyst,AMARO,"São Paulo, São Paulo, Brazil"
2,"Consultant, Data Analytics",Control Risks,"Subprefecture of Sé, São Paulo, Brazil"
3,Engenheiro de Dados,Conquest One,"São Paulo, São Paulo, Brazil"
4,Data Analyst,Zé Delivery,Greater Sao Paulo
5,Analista de BI,Flag Comunicação,"São Paulo, São Paulo, Brazil"
6,Especialista em Planejamento e Análise de Dados,VR Beneficios,"São Paulo, Brazil"
7,Data Analyst - People Analytics,iFood,"Campinas, São Paulo, Brazil"
8,Cientista de dados,Hypera,"São Paulo, São Paulo, Brazil"
9,Data Analyst,Zoox Smart Data,"São Paulo, São Paulo, Brazil"


## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [ ]:
# your code here
